<table>
    <h2>Conversion from Brazilian Standards (SNUC)</h2>
    <tr>
        <th>Proteção Integral (PI)</th>
        <td><a href="https://en.wikipedia.org/wiki/Ecological_station_(Brazil)">Estação ecológica</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Biological_reserve_(Brazil)">Reserva biológica</a></td>
        <td><a href="https://en.wikipedia.org/wiki/National_park">Parque nacional</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Natural_monument">Monumento natural</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Wildlife_refuge_(Brazil)">Refúgio de vida silvestre</a></td>
    </tr>
    <tr>
        <th>Uso Sustentável (US)</th>
        <td><a href="https://en.wikipedia.org/wiki/Area_of_relevant_ecological_interest_(Brazil)">Área de relevante interesse ecológico</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Private_natural_heritage_reserve_(Brazil)">Reserva particular do patrimônio natural</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Environmental_protection_area_(Brazil)">Área de proteção ambiental</a></td>
        <td><a href="https://en.wikipedia.org/wiki/National_forest_(Brazil)">Floresta nacional</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Sustainable_development_reserve_(Brazil)">Reserva de desenvolvimento sustentável</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Wildlife_reserve_(Brazil)">Reserva de fauna</a></td>
        <td><a href="https://en.wikipedia.org/wiki/Extractive_reserve">Reserva extrativista</a></td>
    </tr>
</table>

<h2>Source of Data</h2>

Populational Data - Source: <a href="https://sidra.ibge.gov.br/tabela/6579">IBGE</a>

Protected Areas - Source: i3Geo - Ministry of the Environment (<a href="http://mapas.mma.gov.br/i3geo/datadownload.htm">i3Geo > Áreas Especiais > Unidades de conservação > Unidades de conservação (todas)</a>)<br>(Direct download link: <a href="http://mapas.mma.gov.br/ms_tmp/ucstodas.shp"> .shp</a>, <a href="http://mapas.mma.gov.br/ms_tmp/ucstodas.shx">.shx</a> and <a href="http://mapas.mma.gov.br/ms_tmp/ucstodas.dbf">.dbf</a>)

<h2>Paramters for script execution</h2>

This script has two modes for execution:
<ul>
    <li><b>Lazy Bootstrap</b>: The script will perform some queries and will create a table (with name defined by the user) with the result of the search. In this mode, the user can select all the paramters before excute the database search layer. Pros: Less Disc Space; Cons: Slow
    <li><b>Optimized Bootstrap</b>: The script will perform all possible queries to create a table (named as perfil_uc_all_2). In this mode, the user only select the paramters in the exporting layer, after the execution of the database search layer. Pros: Fast; Cons: More Disc Space (a lot)</li>
</ul>

The paramters are:
<ul>
    <li><b>Type of protected areas;</b></li>
    <li><b>Year of creation;</b></li>
    <li><b>Distance</b> between the city and the protected area;</li>
    <li><b>Name</b> of the data;</li>
</ul>

>Mode of Running
><br>0 = Lazy Bootstrap
><br>1 = Optimized Bootstrap

In [20]:
Enum_Modes = {
    "Lazy" : 0,
    "Optimized" : 1,
}

#You can either use the numbers, or the alias as below
mode = Enum_Modes['Lazy']

>About the requests

>>Tip: If you want to search more than one type of protected area (i.e: PI and US), write as following:
<b>grupo_UC = ['PI','US']</b>
<a href="https://www.mma.gov.br/areas-protegidas/unidades-de-conservacao/categorias.html">Categories of PA by SNUC (Brazil-Standard)</a>

In [21]:
#Type of protected area
grupo_UC = ['PI']

>>Tip: in this paramters, the first value is the minimum year of creation, and the second, the maximum. If you want to search all protected areas created in 2008, write as following:
<b>ano_criação = ('2008','2008')</b>

In [22]:
#creating year of protected area
#ano_criação = ('2005','2015') 
ano_criação = ('0000','9999') 

>Maximum distance between two PA

In [23]:
distancia = "50"

>About database

In [24]:
#Table that storage the results of search in the database
#tabela_resultados = 'perfil_uc_2005_2015_2'
tabela_resultados = 'perfil_uc_all'

In [25]:
#user/password
conexao_banco = ('administrador-banco','senha')

>About the output data

In [26]:
#name of file that contains all data processed (it will be always a .csv file)
arquivo_exportado = 'results.csv'

The algorithm

In [27]:
#mode change
Optimized_params = None
if(mode == Enum_Modes['Optimized']):
    Optimized_params = {
        "ano_criação":(int(ano_criação[0]),int(ano_criação[1])),
        "distancia":float(distancia)
    }
    
    ano_criação = ('0000','9999') 
    distancia = "100000"
    tabela_resultados = 'perfil_uc_all_3'

In [28]:
#dependencies
import psycopg2
import time
import statistics
import os
import pandas

In [29]:
#routine to fetch the query and create a list with all the results
def fetch_query(query):
	cur.execute(query)
	try:
		ret = cur.fetchall()
		return [p[0] for p in ret]
	except:
		print("fail!")
		return []

Database stuff

In [30]:
#create the database(DB) connection
con = psycopg2.connect(host='localhost', database='postgres',
user=conexao_banco[0], password=conexao_banco[1])
cur = con.cursor()

#queries
grupos_procurados = '('+'OR'.join([" grupo4 = '"+g+"'" for g in grupo_UC])+')'
unidades_conservacao = "select nome_uc1 from \"br.domicilios.mma\".ucstodas_final where "+grupos_procurados+" and ano_cria6 >= '"+ano_criação[0]+"' and ano_cria6 <= '"+ano_criação[1]+"'"
cidades_grandes = "select nome_uc1 from \"br.domicilios.mma\".ucstodas_final where "+grupos_procurados+" and ano_cria6 >= '"+ano_criação[0]+"' and ano_cria6 <= '"+ano_criação[1]+"'"

Queries stuff

In [31]:
#Get all Primary Keys (PK) from the Protected Areas (PA) (The PKs on this table is the name of each PA)
recset = fetch_query(unidades_conservacao)
#list of queries to be done
queries = []
#list of execution time of each query (we use this for the statistics)
tempo = []
#progress of the queries (list of queries that was already done)
ja_feito = []

In [32]:
#Load all queries that already executed (if is the first time, this file is empty,so it will read nothing)
try:
    with open("uc_feitas.txt","r",encoding='utf-8') as f:
        for line in f:
            print(line[:-1],"already processed")
            ja_feito.append(line[:-1])
except FileNotFoundError:
    #create the file and leave it in blank
    open('uc_feitas.txt','w+',encoding='utf-8').close()

PARQUE ESTADUAL CHANDLESS already processed
REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUNÃ E DO PADRE already processed
RESERVA BIOLÓGICA DO PARAZINHO already processed
PARQUE NATURAL MUNICIPAL DO CANCãO already processed
PARQUE ESTADUAL SERRA DO ARACÁ already processed
PARQUE ESTADUAL GUARIBA already processed
PARQUE ESTADUAL RIO NEGRO SETOR NORTE already processed
PARQUE ESTADUAL SUMAÚMA already processed
PARQUE ESTADUAL SUCUNDURI already processed
RESERVA BIOLÓGICA MORRO DOS SEIS LAGOS already processed
PARQUE ESTADUAL RIO NEGRO SETOR SUL already processed
PARQUE ESTADUAL DO MATUPIRI already processed
PARQUE ESTADUAL DAS SETE PASSAGENS already processed
PARQUE ESTADUAL PONTA DA TULHA already processed
MONUMENTO NATURAL PENíNSULA DA SIRIBINHA already processed
PARQUE NATURAL MUNICIPAL DA MACAQUEIRAS already processed
MUNUMENTO NATURAL DOS CANIONS DO SUBAÉ already processed
ESTAÇÃO ECOLÓGICA DO RIO PRETO already processed
PARQUE ESTADUAL DO MORRO DO CHAPÉU already processed
PARQUE NA

Execute the search query for each PA's ID that was selected before

In [33]:
try:
    cur.execute("SELECT * from \"br.domicilios.mma\"."+tabela_resultados)
    con.commit()
except psycopg2.errors.UndefinedTable:
    con.commit()
    print('Creating the table...')
    cur.execute("CREATE TABLE \"br.domicilios.mma\"."+tabela_resultados+"(\
            gid INTEGER,\
           geocodigo VARCHAR (17),\
           uf VARCHAR (19),\
           municipio VARCHAR (33),\
           habitantes numeric,\
           nome_uc VARCHAR (255),\
           ano_cria VARCHAR (255),\
           distancia_ VARCHAR (21),\
           geom GEOMETRY\
               )")
    con.commit()


In [34]:
#for each PA
for (i,rec) in enumerate(recset):
	print("Fetching %s..."%rec)
    #skip the already processed PAs
	if(rec in ja_feito):
		print(rec,'already done. Skipping...')
		continue
	start = time.time()
	ano_cria = fetch_query("(SELECT b.ano_cria6 FROM \"br.domicilios.mma\".ucstodas_final b where b.nome_uc1 = \'"+rec+"\')")[0]    
        
	#Execute the query as our paramenters that was set before
	cur.execute("INSERT INTO \"br.domicilios.mma\"."+tabela_resultados+"\
		SELECT a.gid, a.geocodigo, \
       CASE\
        WHEN (a.estado = '11' )  THEN 'rondônia'\
        WHEN (a.estado = '12' )  THEN 'acre'\
        WHEN (a.estado = '13' )  THEN 'amazonas'\
        WHEN (a.estado = '14' )  THEN 'roraima'\
        WHEN (a.estado = '15' )  THEN 'pará'\
        WHEN (a.estado = '16' )  THEN 'amapá'\
	        WHEN (a.estado = '17' )  THEN 'tocantins'\
	        WHEN (a.estado = '21' )  THEN 'maranhão'\
	        WHEN (a.estado = '22' )  THEN 'piauí'\
	        WHEN (a.estado = '23' )  THEN 'ceará'\
	        WHEN (a.estado = '24' )  THEN 'rio grande do norte'\
	        WHEN (a.estado = '25' )  THEN 'paraíba'\
	        WHEN (a.estado = '26' )  THEN 'pernanbuco'\
	        WHEN (a.estado = '27' )  THEN 'alagoas'\
	        WHEN (a.estado = '28' )  THEN 'sergipe'\
	        WHEN (a.estado = '29' )  THEN 'bahia'\
	        WHEN (a.estado = '31' )  THEN 'minas gerais'\
	        WHEN (a.estado = '32' )  THEN 'espírito santo'\
	        WHEN (a.estado = '33' )  THEN 'rio de janeiro'\
	        WHEN (a.estado = '35' )  THEN 'são paulo'\
	        WHEN (a.estado = '41' )  THEN 'paraná'\
	        WHEN (a.estado = '42' )  THEN 'santa catarina'\
	        WHEN (a.estado = '43' )  THEN 'rio grande do sul'\
	        WHEN (a.estado = '50' )  THEN 'mato grosso do sul'\
	        WHEN (a.estado = '51' )  THEN 'mato grosso'\
	        WHEN (a.estado = '52' )  THEN 'goiás'\
	        WHEN (a.estado = '53' )  THEN 'distrito federal'\
	        WHEN (a.estado = '00' )  THEN 'chico mendes'\
	    END AS UF\
	, a.nome as municipio, a.habitantes, '"+rec+"',"+ano_cria+",\
	       to_char(ST_DistanceSphere(\
	                    a.geom, (SELECT b.geom FROM \"br.domicilios.mma\".ucstodas_final b where b.nome_uc1 = \'"+rec+"\')\
	                  ) /1000,'99999999999999999D99') AS \"distancia_km\", a.geom\
	  FROM \"br.domicilios.mma\".municipios_perfil_parsec a\
		WHERE \
		    (ST_DistanceSphere(a.geom, (SELECT b.geom FROM \"br.domicilios.mma\".ucstodas_final b where b.nome_uc1 = \'"+rec+"\')) /1000) <= "+distancia+"  \
        ;")
	#apply the modifications
	con.commit()    
	#The file below stores the PKs of all the processed queries, 
	#in order to avoid duplicate cases if this script needs
	#to restart all the search.
	f = open("uc_feitas.txt","a+",encoding='utf-8')
	f.write(rec+"\n")
	f.close()
	fim = time.time()
    
	
	#Note: Since all this process have a great cost
	#in terms of computer instructions, this could 
	#request a considerable amount of time in 
	#average computers. 

	#Don't be scarred if the remaning time value reaches a high value :)
	#(the time of each query execution is about 2 minutes at Core i7-4790 3.60Ghz)
	
	#calculate and report the statistics about the program
	tempo.append((fim - start))
	v = int(statistics.mean(tempo))*(len(recset)-i)
	print("%d/%d Done. About "%(i+1,len(recset)),int(v/3600)," Hours",int(v%3600)/60,"Minutes left")
    

Fetching PARQUE ESTADUAL CHANDLESS...
PARQUE ESTADUAL CHANDLESS already done. Skipping...
Fetching REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUNÃ E DO PADRE...
REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUNÃ E DO PADRE already done. Skipping...
Fetching RESERVA BIOLÓGICA DO PARAZINHO...
RESERVA BIOLÓGICA DO PARAZINHO already done. Skipping...
Fetching PARQUE NATURAL MUNICIPAL DO CANCãO...
PARQUE NATURAL MUNICIPAL DO CANCãO already done. Skipping...
Fetching PARQUE ESTADUAL SERRA DO ARACÁ...
PARQUE ESTADUAL SERRA DO ARACÁ already done. Skipping...
Fetching PARQUE ESTADUAL GUARIBA...
PARQUE ESTADUAL GUARIBA already done. Skipping...
Fetching PARQUE ESTADUAL RIO NEGRO SETOR NORTE...
PARQUE ESTADUAL RIO NEGRO SETOR NORTE already done. Skipping...
Fetching PARQUE ESTADUAL SUMAÚMA...
PARQUE ESTADUAL SUMAÚMA already done. Skipping...
Fetching PARQUE ESTADUAL SUCUNDURI...
PARQUE ESTADUAL SUCUNDURI already done. Skipping...
Fetching RESERVA BIOLÓGICA MORRO DOS SEIS LAGOS...
RESERVA BIOLÓGICA M

Prepare and export the reults into a csv file

>Export the raw table into csv data

In [35]:
#get the current directory
curr_directory = os.getcwd()

#ensure the csv format
if(arquivo_exportado[-4:] != '.csv'):
    arquivo_exportado += '.csv'

#path to the file
file_name = curr_directory+"/"+arquivo_exportado

#file that contains some cache from our operations
temp_file_name = file_name+'.temp'

#list of columns to save
columns_save = ['gid', 'geocodigo', 'uf', 'municipio', 'habitantes', 'nome_uc', 'ano_cria', 'distancia_']

#export all data to temporary file
with open(temp_file_name,'w+',encoding='latin-1') as f:
    f.write(';'.join(columns_save)+'\n')
    cur.copy_to(f, '"br.domicilios.mma".'+tabela_resultados, columns=tuple(columns_save),sep=';')
    con.commit()

>Change the table columns into: 
<br><b>GID</b>: the ID
<br><b>Geocodigo</b>: Geocode based on IBGE rules
<br><b>UF</b>: Unidades Federativas (Federative Unites, the Subdivisions of Brazil)
<br><b>Municipio</b>: Municipality
<br><b>Habitantes</b>: size of Population
<br><b>UCs_num</b>: Number of protected areas around the Municipality
<br><b>UCs_nome</b>: Name of each protected areas.
<br><b>UCs_anos</b>: Year of creation for each protected areas.
<br><b>Distancias_km</b>: distances (in km), separated by ',' (comma), for each Protected Area, in the same order of the column <b>UCs_nome</b>
<br>
<br>Is important to note, the value of column <b>UCs_num</b> is the size of the column <b>UCs_nome</b> and <b>Distancias_</b>

In [36]:
read_content = pandas.read_csv(temp_file_name, delimiter = ';', encoding='latin-1')

#store GIDs, Distances, and PA's Names
distances_GIDs = {}
names_GIDs = {}
years_GIDs = {}
for gid,distance,name,year in zip(read_content['gid'],read_content['distancia_'],read_content['nome_uc'],read_content['ano_cria']):
    if(gid in distances_GIDs):
        distances_GIDs[gid].append(distance.replace(' ','').replace(',','.'))
    else:
        distances_GIDs[gid] = [distance.replace(' ','').replace(',','.')]
    if(gid in names_GIDs):
        names_GIDs[gid].append(name)
    else:
        names_GIDs[gid] = [name]
    if(gid in years_GIDs):
        years_GIDs[gid].append(str(year))
    else:
        years_GIDs[gid] = [str(year)]

processed_GIDs = []
with open(file_name,'w+',encoding='latin-1') as f:
    #header of csv
    f.write('gid;geocodigo;uf;municipio;habitantes;UCs_num;UCs_nome;UCs_anos;distancias_km\n')
    for content in zip(read_content['gid'],read_content['geocodigo'],read_content['uf'],read_content['municipio'],read_content['habitantes']):
        if(not content[0] in processed_GIDs):
            #precisam estar a pelo menos 50km de distancia de uma UC
            if(Optimized_params):
                if(any([float(d) >= Optimized_params["distancia"] for d in distances_GIDs[content[0]]])):
                    continue
                if(any([(int(d) >= Optimized_params["ano_cria"][1] and int(d) <= Optimized_params["ano_cria"][0]) for d in year_GIDs[content[0]]])):
                    continue
            processed_GIDs.append(content[0])
            c = [str(s) for s in content]
            column_UCs_num = str(len(distances_GIDs[content[0]]))
            column_UCs_nome = ','.join(names_GIDs[content[0]])
            column_UCs_anos = ','.join(years_GIDs[content[0]])
            column_Distancias_km = ','.join(distances_GIDs[content[0]])
            buffer = ';'.join(c)+';'+column_UCs_num+';'+column_UCs_nome+';'+column_UCs_anos+';'+column_Distancias_km+'\n'
            f.write(buffer)

print('Export the output data into',file_name)

Export the output data into C:\Users\Guilherme/results.csv


The function below gather all municipalities from array 'municipios', as the cited conditions

In [53]:
def export_csv_municipios(municipios,nome_arquivo):
    
    #ensure they are all lowercase
    municipios = [name.lower().replace("'","''") for name in municipios]
    
    file_name = curr_directory+"/"+arquivo_exportado
    
    read_content = pandas.read_csv(file_name, delimiter = ';', encoding='latin-1')
    
    municipios_query = []
    ucs_names = []
    
    file_name = curr_directory+'/'+nome_arquivo
    with open(file_name,'w+',encoding='latin-1') as f:
        #header of csv
        f.write('gid;geocodigo;uf;municipio;habitantes;UCs_num;UCs_nome;UCs_anos;distancias_km\n')
        for content in zip(read_content['gid'],read_content['geocodigo'],read_content['uf'],read_content['municipio'],read_content['habitantes'],read_content['UCs_num'],read_content['UCs_nome'],read_content['UCs_anos'],read_content['distancias_km']):
            if(content[3] in municipios):
                municipios_query.append(content[3])
                ucs_names += content[6].split(', ')
                c = [str(s) for s in content]
                buffer = ';'.join(c)+'\n'
                f.write(buffer)
    
    print("Query municipios\n\n")
    print('SELECT * FROM "br.domicilios.mma".perfil_uc_2005_2015_2 where municipio = '+' OR municipio = '.join(["'"+m+"'" for m in municipios_query]))
    print("\n"*5)
    print("Query PAs\n\n")
    print('SELECT * FROM "br.domicilios.mma".ucstodas_final where nome_uc1 = '+' OR nome_uc1 = '.join(["'"+m+"'" for m in ucs_names]))
    print('Export the output data into',file_name)

<h2>Example of Operation</h2>
<br>Selecting the Municipalities from <a href="https://pt.wikipedia.org/wiki/Vale_do_Ribeira">Vale do Ribeira</a>

In [54]:
Vale_do_Ribeira = {
    'Alto Ribeira'  :['Barra do Chapéu','Itapirapuã Paulista','Apiaí','Itaóca','Iporanga','Ribeira','Adrianópolis'],
    'Baixo Ribeira' :['Apiaí','Iporanga','Eldorado','Sete Barras'],
    'Rio Ribeira de Iguape' :['Registro','Pariquera-Açu','Iguape'],
    'Alto Juquiá' : ['São Lourenço da Serra','Juquitiba','Tapiraí'],
    'Médio Juquiá' : ['Tapiraí','Juquiá','Miracatu'],
    'Baixo Juquiá' : ['Juquiá','Tapiraí','Sete Barras'],
    'Rio São Lourenço' : ['Miracatu','Pedro de Toledo','Juquiá'],
    'Rio Itariri': ['Itariri','Pedro de Toledo'],
    'Rio Una da Aldeia' : ['Iguape'],
    'Rio Pardo' : ['Barra do Turvo'],
    'Rio Jacupiranga' : ['Jacupiranga','Pariquera-Açu','Cajati','Registro'],
    'Vertente Marítima Sul' : ['Cananéia','Ilha Comprida'],
    'Vertente Marítima Norte' : ['Iguape']
}


municipios = []
for região in Vale_do_Ribeira:
    municipios += Vale_do_Ribeira[região]

#do the filter
export_csv_municipios(municipios,'vale_do_ribeira.csv')

pandas.read_csv('vale_do_ribeira.csv', delimiter = ';', encoding='latin-1')

Query municipios


SELECT * FROM "br.domicilios.mma".perfil_uc_2005_2015_2 where municipio = 'tapiraí' OR municipio = 'barra do turvo' OR municipio = 'ilha comprida' OR municipio = 'adrianópolis' OR municipio = 'pedro de toledo' OR municipio = 'itapirapuã paulista' OR municipio = 'iporanga' OR municipio = 'itaóca' OR municipio = 'ribeira' OR municipio = 'barra do chapéu' OR municipio = 'tapiraí'






Query PAs


SELECT * FROM "br.domicilios.mma".ucstodas_final where nome_uc1 = 'PARQUE NACIONAL DA SERRA DA CANASTRA,PARQUE ESTADUAL CAMPOS ALTOS' OR nome_uc1 = 'PARQUE NACIONAL DO SUPERAGUI,ESTAÇÃO ECOLÓGICA DE GUARAQUEÇABA,RESERVA BIOLÓGICA BOM JESUS,ESTAÇÃO ECOLÓGICA DOS TUPINIQUINS,ESTAÇÃO ECOLÓGICA DA ILHA DO MEL,PARQUE ESTADUAL DE CAMPINHOS,PARQUE ESTADUAL NASCENTES DO PARANAPANEMA,PARQUE ESTADUAL INTERVALES,PARQUE ESTADUAL CAVERNA DO DIABO,PARQUE ESTADUAL  DA ILHA DO CARDOSO,ESTAÇÃO ECOLÓGICA DE XITUÉ,PARQUE ESTADUAL DA CAMPINA DO ENCANTADO,PARQUE NATURAL MUNICIPAL MORRO DO OURO,PAR

,gid,geocodigo,uf,municipio,habitantes,UCs_num,UCs_nome,UCs_anos,distancias_km
0,1416,31-68200-00-00-00,minas gerais,tapiraí,1875,2,"PARQUE NACIONAL DA SERRA DA CANASTRA,PARQUE ES...","1972,2004","32.71,6.85"
1,1506,35-05401-00-00-00,são paulo,barra do turvo,7659,16,"PARQUE NACIONAL DO SUPERAGUI,ESTAÇÃO ECOLÓGICA...","1989,1982,2012,1986,1982,1960,2012,1995,2008,1...","19.76,17.06,10.95,36.44,46.56,47.57,43.65,14.9..."
2,1579,35-20426-00-00-00,são paulo,ilha comprida,11166,13,"PARQUE NACIONAL DO SUPERAGUI,ESTAÇÃO ECOLÓGICA...","1989,1982,1986,1986,2008,1962,1994,1978,2008,1...","21.83,42.30,11.11,7.86,42.78,1.09,16.53,2.63,1..."
3,1771,41-00202-00-00-00,paraná,adrianópolis,5919,11,"PARQUE NACIONAL DO SUPERAGUI,ESTAÇÃO ECOLÓGICA...","1989,1982,2012,1960,2004,2008,1995,2008,1958,1...","48.26,37.10,26.04,13.39,14.03,.32,20.88,.00,2...."
4,1666,35-37206-00-00-00,são paulo,pedro de toledo,11331,13,"ESTAÇÃO ECOLÓGICA DOS TUPINIQUINS,PARQUE ESTAD...","1986,1982,1986,2012,2012,2012,2013,1977,2012,2...","17.93,48.76,.00,42.11,36.70,38.71,18.92,.00,40..."
5,1590,35-22653-00-00-00,são paulo,itapirapuã paulista,4241,5,"PARQUE ESTADUAL VALE DO CóDO,PARQUE ESTADUAL D...","2007,1960,2004,1985,1958","41.58,37.98,28.67,38.67,39.09"
6,1585,35-21200-00-00-00,são paulo,iporanga,4218,10,"PARQUE ESTADUAL DE CAMPINHOS,PARQUE NATURAL MU...","1960,2004,1982,2012,1995,2008,1958,1957,2008,2008","47.40,5.02,29.86,7.36,.00,.00,.00,.00,12.11,41.18"
7,1589,35-22158-00-00-00,são paulo,itaóca,3328,7,"PARQUE ESTADUAL DE CAMPINHOS,PARQUE NATURAL MU...","1960,2004,1995,2008,1958,1957,2008","42.57,4.27,29.66,9.65,.00,48.51,32.96"
8,1689,35-42800-00-00-00,são paulo,ribeira,3340,7,"PARQUE ESTADUAL DE CAMPINHOS,PARQUE NATURAL MU...","1960,2004,2008,1995,2008,1985,1958","37.52,7.41,43.57,41.89,27.33,49.14,16.85"
9,1505,35-05351-00-00-00,são paulo,barra do chapéu,5724,6,"PARQUE ESTADUAL VALE DO CóDO,PARQUE ESTADUAL I...","2007,1995,2008,1985,2004,1958","42.65,43.39,35.32,28.01,10.69,21.02"


><a href="https://www.mma.gov.br/informma/item/4278-mma-define-municipios-da-bacia-do-sao-francisco"> Macrozoneamento da bacia do rio São Francisco </a>

In [56]:
bacia_rio_sao_francisco = [
    'Campo Formoso','Jacobina','Miguel Calmon','Mirangaba','Morro do Chapéu','Ourolândia','Umburanas','Várzea Nova',
'América Dourada','Barra do Mendes','Barro Alto','Bonito','Cafarnaum','Canarana','Central','Ibipeba','Ibititá','Irecê',
'João Dourado','Jussara','Lapão','Mulungu do Morro','Presidente Dutra','São Gabriel','Souto Soares','Uibaí','Amparo de São Francisco',
'Aquidabã','Brejo Grande','Canhoba','Capela','Cedro de São João','Feira Nova','Gararu','Gracho Cardoso','Ilha das Flores','Itabi',
'Japaratuba','Japoatã','Malhada dos Bois','Monte Alegre de Sergipe','Muribeca','Neópolis','Nossa Senhora da Glória','Nossa Senhora de Lourdes',
'Pacatuba','Pirambu','Porto da Folha','Poço Redondo','Propriá','Santana do São Francisco','Idreg','São Francisco','Telha','Arapiraca','Batalha',
'Belo Monte','Cacimbinhas','Campo Grande','Canapi','Carneiros','Coruripe','Craíbas','Dois Riachos','Estrela de Alagoas','Feira Grande',
'Feliz Deserto','Girau do Ponciano','Igaci','Igreja Nova','Inhapi','Jacaré dos Homens','Jaramataia','Junqueiro','Lagoa da Canoa',
'Limoeiro de Anadia','Major Isidoro','Maravilha','Minador do Negrão','Monteirópolis','Olho d\'água das Flores','Olho d\'água Grande',
'Olivença','Ouro Branco','Palestina','Pão de Açúcar','Penedo','Piaçabuçu','Porto Real do Colégio','Poço das Trincheiras','Santana do Ipanema',
'São Brás','São José da Tapera','São Sebastião','Senador Rui Palmeira','Teotônio Vilela','Traipu','Água Branca','Canindé de São Francisco',
'Delmiro Gouveia','Glória','Inajá','Jatobá','Jeremoabo','Mata Grande','Olho d\'água do Casado','Pariconha','Paulo Afonso','Pedro Alexandre',
'Petrolândia','Piranhas','Santa Brígida','Tacaratu','Águas Belas','Afogados da Ingazeira','Alagoinha','Arcoverde','Betânia','Bom Conselho',
'Brejinho','Buíque','Caetés','Calumbi','Carnaíba','Custódia','Flores','Iati','Ibimirim','Iguaraci','Ingazeira','Itaíba','Itapetim',
'Manari','Paranatama','Pedra','Pesqueira','Quixaba','Saloá','Santa Cruz da Baixa Verde','Santa Terezinha','São José do Belmonte',
'São José do Egito','Serra Talhada','Sertânia','Solidão','Tabira','Triunfo','Tupanatinga','Tuparetama','Venturosa','Abaré','Belém de São Francisco',
'Cabrobó','Carnaubeira da Penha','Cedro','Chorrochó','Floresta','Itacuruba','Macururé','Mirandiba','Moreilândia','Parnamirim','Rodelas',
'Salgueiro','Serrita','Terra Nova','Verdejante','Araripina','Bodocó','Exu','Granito','Ipubi','Ouricuri','Santa Cruz','Santa Filomena',
'Trindade','Afrânio','Campo Alegre de Lourdes','Casa Nova','Curaçá','Dormentes','Jaguarari','Juazeiro','Lagoa Grande','Orocó','Petrolina',
'Pilão Arcado','Remanso','Santa Maria da Boa Vista','Sento Sé','Sobradinho','Uauá','Barra','Buritirama','Gentio do Ouro','Itaguaçu da Bahia',
'Xique-Xique','Érico Cardoso','Bom Jesus da Lapa','Boninal','Boquira','Botuporã','Brejolândia','Brotas de Macaúbas','Canópolis',
'Caturama','Ibipitanga','Ibitiara','Ibotirama','Ipupiara','Macaúbas','Morpará','Muquém de São Francisco','Novo Horizonte','Oliveira dos Brejinhos',
'Paramirim','Paratinga','Piatã','Rio de Contas','Rio do Pires','Santa Maria da Vitória','Santana','São Félix do Coribe','Seabra',
'Serra do Ramalho','Serra Dourada','Sítio do Mato','Tabocas do Brejo Velho','Tanque Novo','Angical','Baianópolis','Barreiras',
'Catolândia','Cocos','Coribe','Correntina','Cotegipe','Cristópolis','Formosa do Rio Preto','Jaborandi','Luís Eduardo Magalhães','Mansidão',
'Riachão das Neves','Santa Rita de Cássia','São Desidério','Wanderley','Caetité','Candiba','Carinhanha','Catuti','Espinosa','Feira da Mata',
'Gameleiras','Guanambi','Igaporã','Iuiú','Jacaraci','Jaíba','Janaúba','Malhada','Mamonas','Matias Cardoso','Matina','Mato Verde','Monte Azul',
'Mortugaba','Nova Porteirinha','Pai Pedro','Palmas de Monte Alto','Pindaí','Porteirinha','Riacho de Santana','Riacho dos Machados',
'Sebastião Laranjeiras','Serranópolis de Minas','Urandi','Varzelândia','Verdelândia','Bonito de Minas','Cônego Marinho','Ibiracatu','Itacarambi',
'Januária','Juvenília','Manga','Miravânia','Montalvânia','Pedras de Maria da Cruz','São João das Missões','Arinos','Bonfinópolis de Minas',
'Brasília','Buritis','Cabeceira Grande','Cabeceiras','Brasília','Cristalina','Dom Bosco','Formosa','Formoso','Guarda-Mor','Natalândia',
'Paracatu','Riachinho','Unaí','Uruana de Minas','Arapuá','Carmo do Paranaíba','João Pinheiro','Lagamar','Lagoa Formosa','Lagoa Grande',
'Matutina','Patos de Minas','Presidente Olegário','Rio Paranaíba','Santa Rosa da Serra','Patos de Minas','São Gotardo','Tiros','Varjão de Minas',
'Vazante','Chapada Ga·cha','São Gonþalo do Abaeté','Bocaiúva','Brasilândia de Minas','Brasília de Minas','Buenópolis','Buritizeiro','Campo Azul',
'Capitão Enéas','Claro dos Poções','Coração de Jesus','Engenheiro Navarro','Francisco Dumont','Francisco Sá','Glaucilândia','Guaraciama','Ibiaí',
'Icaraí de Minas','Japonvar','Jequitaí','Joaquim Felício','Juramento','Lagoa dos Patos','Lassance','Lontra','Luislândia','Mirabela','Montes Claros',
"Olhos d'água",'Patis','Pintópolis','Pirapora','Ponto Chique','Santa Fé de Minas','São Francisco','São João da Lagoa','São João da Ponte','São João do Pacuí',
'São Romão','Ubaí','Urucuia','Várzea da Palma','Abaeté','Belo Horizonte','Araújos','Arcos','Augusto de Lima','Baldim','Bambuí',
'Belo Horizonte','Belo Vale','Betim','Biquinhas','Bom Despacho','Bonfim','Brumadinho','Araþaí','Cachoeira da Prata','Caetanópolis',
'Caeté','Campos Altos','Capim Branco','Carmo da Mata','Carmo do Cajuru','Carmópolis de Minas','Casa Grande','Cedro do Abaeté','Cláudio',
'Conceição do Pará','Conceição do Mato Dentro','Confins','Congonhas','Congonhas do Norte','Conselheiro Lafaiete',
'Contagem','Cordisburgo','Corinto','Córrego Danta','Córrego Fundo','Cristiano Otoni','Crucilândia','Curvelo','Datas',
'Desterro de Entre Rios','Diamantina','Divinópolis','Dores do Indaiá','Doresópolis','Entre Rios de Minas','Esmeraldas',
'Estrela do Indaiá','Felixlândia','Florestal','Formiga','Fortuna de Minas','Funilândia','Gouveia','Ibirité','Igarapé',
'Igaratinga','Iguatama','Inhaúma','Inimutaba','Itaúna','Itabirito','Itaguara','Itapecerica','Belo Horizonte',
'Itaverava','Jaboticatubas','Japaraíba','Itatiaiuþu','Jeceaba','Jequitibá','Juatuba','Lagoa da Prata','Lagoa Dourada','Lagoa Santa',
'Leandro Ferreira','Luz','Maravilhas','Martinho Campos','Mateus Leme','Matozinhos','Medeiros','Moeda','Moema','Monjolos','Morada Nova de Minas',
'Morro da Garça','Mário Campos','Nova Lima','Nova Serrana','Nova União','Oliveira','Onça de Pitangui','Ouro Branco','Ouro Preto',
'Paineiras','Pains','Papagaios','Paraopeba','Pará de Minas','Passa Tempo','Pedra do Indaiá','Pedro Leopoldo','Pequi','Perdigão',
'Piedade dos Gerais','Pimenta','Piracema','Pitangui','Piumhi','Pompéu','Presidente Juscelino','Presidente Kubitschek','Prudente de Morais',
'Quartel Geral','Queluzito','Raposos','Resende Costa','Ribeirão das Neves','Rio Acima','Rio Manso','Sabará',
'Santa Luzia','Santana de Pirapama','Santana do Riacho','Santo Antônio do Monte','Santo Hipólito',
'São Brás do Suaçuí','São Francisco de Paula','São Gonçalo do Pará','São Joaquim de Bicas','São José da Lapa',
'São José da Varginha','São Roque de Minas','São Sebastião do Oeste','Sarzedo','Serra da Saudade','Sete Lagoas',
'Tapiraí','Belo Horizonte','Três Marias','Vargem Bonita','Vespasiano','Taquaraþu de Minas',
]

#do the filter
export_csv_municipios(bacia_rio_sao_francisco,'rio_sao_francisco.csv')
pandas.read_csv('rio_sao_francisco.csv', delimiter = ';', encoding='latin-1')

Query municipios


SELECT * FROM "br.domicilios.mma".perfil_uc_2005_2015_2 where municipio = 'carneiros' OR municipio = 'maravilha' OR municipio = 'ouro branco' OR municipio = 'pariconha' OR municipio = 'mulungu do morro' OR municipio = 'iuiú' OR municipio = 'sebastião laranjeiras' OR municipio = 'gameleiras' OR municipio = 'mamonas' OR municipio = 'piranhas' OR municipio = 'itaverava' OR municipio = 'jeceaba' OR municipio = 'moeda' OR municipio = 'queluzito' OR municipio = 'rio acima' OR municipio = 'são brás do suaçuí' OR municipio = 'belo vale' OR municipio = 'bonfim' OR municipio = 'capim branco' OR municipio = 'confins' OR municipio = 'são roque de minas' OR municipio = 'tapiraí' OR municipio = 'vargem bonita' OR municipio = 'florestal' OR municipio = 'nova união' OR municipio = 'rio manso' OR municipio = 'casa grande' OR municipio = 'cristiano otoni' OR municipio = 'crucilândia' OR municipio = 'desterro de entre rios' OR municipio = 'passa tempo' OR municipio = 'piedade dos gerai

,gid,geocodigo,uf,municipio,habitantes,UCs_num,UCs_nome,UCs_anos,distancias_km
0,726,27-01803-00-00-00,alagoas,carneiros,9081,2,REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUN...,"2012,2009","46.04,38.96"
1,738,27-04609-00-00-00,alagoas,maravilha,9163,2,REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUN...,"2012,2009","29.49,35.46"
2,745,27-06109-00-00-00,alagoas,ouro branco,11496,2,REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUN...,"2012,2002","44.82,49.88"
3,747,27-06422-00-00-00,alagoas,pariconha,10533,3,REFÚGIO DE VIDA SILVESTRE DOS MORROS DO CARAUN...,"2012,1984,2009","8.37,49.75,12.19"
4,869,29-22052-00-00-00,bahia,mulungu do morro,10889,2,"PARQUE ESTADUAL DO MORRO DO CHAPÉU,PARQUE NACI...","1998,1985","42.60,33.16"
5,846,29-17334-00-00-00,bahia,iuiú,10994,8,"PARQUE ESTADUAL DA SERRA DOS MONTES ALTOS,REFU...","2010,2010,2007,1988,1999,2000,1973,1998","26.70,24.42,46.17,22.10,46.53,23.75,31.37,1.25"
6,897,29-30006-00-00-00,bahia,sebastião laranjeiras,11434,7,"PARQUE ESTADUAL DA SERRA DOS MONTES ALTOS,REFU...","2010,2010,2007,1988,2000,1973,1998",".00,.00,6.80,34.33,42.22,31.40,21.35"
7,1104,31-27339-00-00-00,minas gerais,gameleiras,5109,9,"PARQUE ESTADUAL DA SERRA DOS MONTES ALTOS,REFU...","2010,2010,1998,2007,1988,2000,2003,1973,1998","37.92,36.86,49.67,.00,33.28,42.20,42.72,21.56,..."
8,1189,31-39250-00-00-00,minas gerais,mamonas,6543,5,"PARQUE ESTADUAL DA SERRA DOS MONTES ALTOS,REFU...","2010,2010,2007,2003,2007","31.65,30.37,.00,41.97,44.22"
9,2772,52-17203-00-00-00,goiás,piranhas,10385,1,PARQUE ESTADUAL SERRA AZUL,1994,43.39


><a href="https://www.mma.gov.br/estruturas/161/_publicacao/161_publicacao03032011023605.pdf">Municípios Rio Parnaiba (page 125)</a>

In [57]:
rio_parnaiba = [
    'Balsas','São Raimundo das Mangabeiras','Riachão',
    'Uruçui','Pastos Bons','Alto Parnaíba','Corrente',
    'Bom Jesus','Monte Alegre do Piauí','Floriano',
    'Barão de Grajaú','Itauira','Picos','Oeiras',
    'São Raimundo Nonato','Teresina','Timon','Crateús',
    'Parnaíba','Piripiri','Campo Maior'
]
export_csv_municipios(rio_parnaiba,'rio_parnaiba.csv')
pandas.read_csv('rio_parnaiba.csv', delimiter = ';', encoding='latin-1')

Query municipios


SELECT * FROM "br.domicilios.mma".perfil_uc_2005_2015_2 where municipio = 'bom jesus' OR municipio = 'riachão' OR municipio = 'bom jesus' OR municipio = 'alto parnaíba' OR municipio = 'monte alegre do piauí' OR municipio = 'bom jesus' OR municipio = 'bom jesus'






Query PAs


SELECT * FROM "br.domicilios.mma".ucstodas_final where nome_uc1 = 'PARQUE NACIONAL DA SERRA GERAL,ESTAÇÃO ECOLÓGICA DE ARACURI-ESMERALDA,PARQUE NACIONAL DE SÃO JOAQUIM,PARQUE NACIONAL DE APARADOS DA SERRA,PARQUE ESTADUAL DO IBITIRIá,PARQUE ESTADUAL DO TAINHAS,RESERVA BIOLÓGICA ESTADUAL DO AGUAÍ' OR nome_uc1 = 'RESERVA BIOLÓGICA GUARIBAS,ESTAçãO ECOLóGICA DO PAU BRASIL,PARQUE ESTADUAL MATA DO PAU FERRO' OR nome_uc1 = 'PARQUE NACIONAL DAS ARAUCÁRIAS,ESTAÇÃO ECOLÓGICA DE MATA PRETA,PARQUE ESTADUAL DE PALMAS,PARQUE ESTADUAL DAS ARAUCÁRIAS' OR nome_uc1 = 'ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,PARQUE NACIONAL DAS NASCENTES DO RIO PARNAIBA,PARQUE ESTADUAL DO JALAPãO,MONUMENTO NATURAL CANYONS E 

,gid,geocodigo,uf,municipio,habitantes,UCs_num,UCs_nome,UCs_anos,distancias_km
0,2227,43-02303-00-00-00,rio grande do sul,bom jesus,11349,7,"PARQUE NACIONAL DA SERRA GERAL,ESTAÇÃO ECOLÓGI...","1992,1981,1961,1959,1975,1975,1983","32.79,38.65,49.30,31.14,.00,17.04,29.90"
1,670,25-12747-00-00-00,paraíba,riachão,3588,3,"RESERVA BIOLÓGICA GUARIBAS,ESTAçãO ECOLóGICA D...","1990,2002,2005","41.70,46.90,41.79"
2,2034,42-02537-00-00-00,santa catarina,bom jesus,3010,4,"PARQUE NACIONAL DAS ARAUCÁRIAS,ESTAÇÃO ECOLÓGI...","2005,2005,2007,2003","33.28,20.45,44.89,28.89"
3,188,21-00501-00-00-00,maranhão,alto parnaíba,11190,4,"ESTACAO ECOLOGICA SERRA GERAL DO TOCANTINS,PAR...","2001,2002,2001,2012","32.69,.00,17.78,26.22"
4,352,22-06605-00-00-00,piauí,monte alegre do piauí,10613,3,"ESTAÇÃO ECOLÓGICA DE URUÇUÍ-UNA,PARQUE NACIONA...","1981,2002,1998","29.94,23.32,26.21"
5,585,25-02201-00-00-00,paraíba,bom jesus,2561,1,MONUMENTO NATURAL VALE DOS DINOSSAUROS,2002,33.91
6,466,24-01701-00-00-00,rio grande do norte,bom jesus,10210,1,PARQUE NATURAL MUNICIPAL DA CIDADE DO NATAL DO...,2006,36.83


End of the queries, close the program

In [41]:
#Close the DB connection
con.close()
print("End :)")
exit(1)

End :)
